In [1]:
from birdset.datamodule import DatasetConfig
from birdset.datamodule.birdset_datamodule import BirdSetDataModule
from birdset.datamodule.components.transforms import BirdSetTransformsWrapper
from transformers import Wav2Vec2Model, HubertModel, Trainer
import torch
from torchmetrics import Accuracy, Precision
import time
import fairseq
from tqdm import tqdm
import lightning as lt
from huggingface_hub import PyTorchModelHubMixin
from lightning.pytorch.loggers import TensorBoardLogger
from torchmetrics.classification.average_precision import AveragePrecision
from torchmetrics.classification.auroc import AUROC
from sklearn.metrics import average_precision_score
import time, os

In [2]:
model_name_base = "facebook/hubert-base-ls960"
model_name_large = "facebook/hubert-large-ll60k"
model_name = model_name_large
basemodel = Wav2Vec2Model.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation=None)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-large-ll60k and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from Model import Modelwrapper

model = Modelwrapper.load_model("D:\\Documents\\Uni\DLL\\Code\models\\Modelwrapper06-10-2024-21-00-09_large50.pt", basemodel)

In [4]:
transforms = BirdSetTransformsWrapper(
        task="multilabel",
        sampling_rate = 16000,
        model_type = 'waveform',
    )

dataset_name = "HSN"
dm = BirdSetDataModule(
    dataset= DatasetConfig(
        data_dir='B:\DLL\Datasets',
        dataset_name= "",
        hf_path='DBD-research-group/BirdSet',
        hf_name=dataset_name,
        n_workers=8,
        val_split=0.2,
        task="multilabel",
        classlimit=500,
        eventlimit=1,
        sampling_rate=16000,
    ),
    transforms=transforms
)

dm.prepare_data()
dm.setup(stage="test")
test_dataloader = dm.test_dataloader()
print(type(test_dataloader.dataset))
print(test_dataloader.dataset)
print(type(test_dataloader))

One-hot-encoding (num_proc=8):   0%|          | 0/11223 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8978 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2245 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/12000 [00:00<?, ? examples/s]

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['filepath', 'labels', 'detected_events', 'start_time', 'end_time'],
    num_rows: 12000
})
<class 'torch.utils.data.dataloader.DataLoader'>


In [5]:
batch = next(iter(test_dataloader))
print(batch["input_values"])

tensor([[[-0.0004, -0.0006, -0.0005,  ...,  0.0022,  0.0021,  0.0023]],

        [[ 0.0016,  0.0024,  0.0023,  ...,  0.0032,  0.0031,  0.0033]],

        [[ 0.0023,  0.0033,  0.0033,  ..., -0.0009, -0.0009, -0.0009]],

        ...,

        [[ 0.0443,  0.0645,  0.0567,  ..., -0.0116, -0.0110, -0.0116]],

        [[-0.0082, -0.0119, -0.0109,  ...,  0.0063,  0.0068,  0.0080]],

        [[ 0.0067,  0.0106,  0.0106,  ..., -0.0522, -0.0486, -0.0508]]])


In [6]:
print(model.get_proba_sigmoid(batch))
print(batch["labels"])

tensor([[0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 